In [1371]:
import pandas as pd
import numpy as np
import torch
import tqdm


In [1372]:
def normalizeCollumns(in_file):
    for i in range(in_file.shape[1]):
        #print(in_file[in_file.columns[i]])
        column=in_file[in_file.columns[i]]
        in_file[in_file.columns[i]]=(column-np.min(column))/(np.max(column)-np.min(column))
        #print(in_file[:,1])
    return in_file

In [1373]:
def get_dataloader(root):
    train_file = pd.read_csv(root)
    #print(train_file)
    in_label =train_file['price_range']
    train_file=train_file.drop(['price_range'],axis=1)

    train_label=torch.zeros([len(in_label),4],dtype=torch.float64)
    for i in range(len(in_label)):
        train_label[i][in_label[i]]=1
        
    normalizeCollumns(train_file)

    return torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.tensor(np.array(train_file)),train_label),batch_size=64,),train_file.shape[1]

In [1374]:
device="cuda" if torch.cuda.is_available() else "cpu"
print("For training used a :", device)

For training used a : cuda


In [1375]:
trainDataloader, countOfParametrs = get_dataloader("/dataset/train.csv")


In [1376]:
class MobileClassificator(torch.nn.Module):
    def __init__(self,countOfParametrs=20):
        super().__init__()
        act_func=torch.nn.ReLU()
        linear_1 = torch.nn.Linear(countOfParametrs,countOfParametrs)
        linear_2= torch.nn.Linear(countOfParametrs,4)

        self.model = torch.nn.Sequential(
            linear_1,act_func,linear_2
        )

    def forward(self,x):
        return self.model(x)
    
model=MobileClassificator().to(device)


In [1377]:
loss_func=torch.nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=1e-2)

In [1378]:
def accruracy(pred,label):
    answer = pred.detach().numpy().argmax(1) == label.numpy().argmax(1) 
    return answer.mean()

In [1379]:
def train(dataloader,model,loss_fn,optimiser):
    loss_val=0
    accruracy_val=0
    for sample in dataloader:
        X,Y=sample
        
        
        X=X.to(device)
        Y=Y.to(device)
        
        X=X.to(torch.float32)
        #Y=Y.unsqueeze(1)
        
        pred=model(X)
        #print(pred.dtype,Y.dtype)
        # for i in range(5):
        #     print(pred[i],"====",Y[i])

        loss=loss_fn(pred,Y)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        loss_val+=loss.item()
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
        # print("accc=",accruracy(pred.cpu(),Y.cpu()))
        # print("loss=",loss.item())
        
    print("avg loss=",loss_val/len(dataloader))
    print("avg accuracy=",accruracy_val/len(dataloader))
    


In [1380]:
def test(dataloader,model):
    accruracy_val=0
    for sample in dataloader:
        X,Y=sample
        
        X=X.to(device)
        Y=Y.to(device)
        
        X=X.to(torch.float32)
        pred=model(X)
        accruracy_val+= accruracy(pred.cpu(),Y.cpu())
    print("accuracy=",accruracy_val/len(dataloader))


In [1381]:
countOfEpoch=20
for epoch in range(countOfEpoch):
    print("Epoch#",epoch+1)
    train(trainDataloader,model,loss_func,optimiser)
    


Epoch# 1
avg loss= 1.3218923055683263
avg accuracy= 0.36279296875
Epoch# 2
avg loss= 1.0318045344756683
avg accuracy= 0.62451171875
Epoch# 3
avg loss= 0.7796790482461802
avg accuracy= 0.728515625
Epoch# 4
avg loss= 0.6402661095853546
avg accuracy= 0.77685546875
Epoch# 5
avg loss= 0.5512309908513089
avg accuracy= 0.81591796875
Epoch# 6
avg loss= 0.48592321062278643
avg accuracy= 0.841796875
Epoch# 7
avg loss= 0.4360719573096503
avg accuracy= 0.86083984375
Epoch# 8
avg loss= 0.39441797183474137
avg accuracy= 0.8798828125
Epoch# 9
avg loss= 0.36128600097401886
avg accuracy= 0.8916015625
Epoch# 10
avg loss= 0.3332627438313409
avg accuracy= 0.90185546875
Epoch# 11
avg loss= 0.3095711505764882
avg accuracy= 0.91259765625
Epoch# 12
avg loss= 0.28856724412335666
avg accuracy= 0.91845703125
Epoch# 13
avg loss= 0.2707549924568866
avg accuracy= 0.9248046875
Epoch# 14
avg loss= 0.25513575510759523
avg accuracy= 0.927734375
Epoch# 15
avg loss= 0.24158304066675385
avg accuracy= 0.92919921875
Epoch# 

In [1382]:
#testDataloader,=get_dataloader("/dataset/test.csv")
#test(testDataloader,model)